In [36]:
import numpy as np

In [37]:
n = 20

In [38]:
Laguerre = {}
for i in range (1, n+1):
    Roots,Weights = np.polynomial.laguerre.laggauss(i)
    Laguerre["Raíces del polinomio de grado: " + str(i)] = Roots.tolist()
a = 0
for i in Laguerre:
    a += 1
    print ("Las raíces del polinomio de grado " + str(i) + " son: " + str((Laguerre[i])))

Las raíces del polinomio de grado Raíces del polinomio de grado: 1 son: [1.0]
Las raíces del polinomio de grado Raíces del polinomio de grado: 2 son: [0.585786437626905, 3.414213562373095]
Las raíces del polinomio de grado Raíces del polinomio de grado: 3 son: [0.4157745567834791, 2.294280360279042, 6.2899450829374794]
Las raíces del polinomio de grado Raíces del polinomio de grado: 4 son: [0.3225476896193924, 1.7457611011583465, 4.536620296921128, 9.395070912301133]
Las raíces del polinomio de grado Raíces del polinomio de grado: 5 son: [0.26356031971814087, 1.4134030591065168, 3.596425771040722, 7.085810005858837, 12.640800844275782]
Las raíces del polinomio de grado Raíces del polinomio de grado: 6 son: [0.2228466041792607, 1.1889321016726226, 2.992736326059314, 5.77514356910451, 9.83746741838259, 15.982873980601703]
Las raíces del polinomio de grado Raíces del polinomio de grado: 7 son: [0.19304367656036225, 1.0266648953391924, 2.567876744950746, 4.900353084526484, 8.18215344456286